In [8]:
from JEFF_Lib import (
    cov_skip_miss, corr_skip_miss, cov_pairwise, corr_pairwise, ewCovar, ewCorr,
    cov_with_different_ew_var_corr, near_psd, _getAplus, _getPS, _getPu, wgtNorm,
    higham_psd, chol_psd, simulate_normal, simulate_pca, FittedModel, fit_normal,
    fit_general_t, general_t_ll, fit_regression_t, return_calculate, VaR_cal, simple_VaR,
    simple_VaR_sim, simple_ES, simple_ES_sim, VaR_ES, Historical_VaR_ES
)

import autograd.numpy as np
import pandas as pd
from autograd.scipy.stats import norm
from autograd import grad
from datetime import date
from scipy.optimize import brentq
from statsmodels.api import OLS, add_constant
from scipy.optimize import minimize

# Problem 1

In [2]:
# Given data
underlying = 151.03
strike = 165
rf = 4.25 / 100
q = 0.53 / 100
b = rf - q
ivol = 0.2

current_date = "2022-03-13"
expiration_date = "2022-04-15"
days = (np.datetime64(expiration_date) - np.datetime64(current_date)).astype(int)

tradingDayYear = 365
ttm = days / tradingDayYear

In [3]:
def gbsm_greek(call, underlying, strike, ttm, rf, b, ivol, include_greeks=False):
    d1 = (np.log(underlying / strike) + (b + ivol**2 / 2) * ttm) / (ivol * np.sqrt(ttm))
    d2 = d1 - ivol * np.sqrt(ttm)
    
    if call:
        price = underlying * np.exp((b - rf) * ttm) * norm.cdf(d1) - strike * np.exp(-rf * ttm) * norm.cdf(d2)
    else:
        price = strike * np.exp(-rf * ttm) * norm.cdf(-d2) - underlying * np.exp((b - rf) * ttm) * norm.cdf(-d1)
    
    result = {"price": price}
    
    if include_greeks:
        gamma = norm.pdf(d1) * np.exp((b - rf) * ttm) / (underlying * ivol * np.sqrt(ttm))
        vega = underlying * np.exp((b - rf) * ttm) * norm.pdf(d1) * np.sqrt(ttm)
        
        if call:
            delta = np.exp((b - rf) * ttm) * norm.cdf(d1)
            
            theta = (-underlying * np.exp((b - rf) * ttm) * norm.pdf(d1) * ivol / (2 * np.sqrt(ttm))
                     - (b - rf) * underlying * np.exp((b - rf) * ttm) * norm.cdf(d1)
                     - rf * strike * np.exp(-rf * ttm) * norm.cdf(d2))
            
            rho = ttm * strike * np.exp(-rf * ttm) * norm.cdf(d2)
            
            cRho = ttm * underlying * np.exp((b - rf) * ttm) * norm.cdf(d1)
        else:
            delta = np.exp((b - rf) * ttm) * (norm.cdf(d1) - 1)
            
            theta = (-underlying * np.exp((b - rf) * ttm) * norm.pdf(d1) * ivol / (2 * np.sqrt(ttm))
                     + (b - rf) * underlying * np.exp((b - rf) * ttm) * norm.cdf(-d1)
                     + rf * strike * np.exp(-rf * ttm) * norm.cdf(-d2))
            
            rho = -ttm * strike * np.exp(-rf * ttm) * norm.cdf(-d2)
            
            cRho = -ttm * underlying * np.exp((b - rf) * ttm) * norm.cdf(-d1)
        
        result.update({
            "delta": delta,
            "gamma": gamma,
            "vega": vega,
            "theta": theta,
            "rho": rho,
            "cRho": cRho
        })
    
    return result

In [4]:
# Calculate GBSM values for call and put
bsm_call = gbsm_greek(True, underlying, strike, ttm, rf, b, ivol, include_greeks=True)
bsm_put = gbsm_greek(False, underlying, strike, ttm, rf, b, ivol, include_greeks=True)

In [5]:
# Create a DataFrame for display
bsm_results = pd.DataFrame({
    "Greeks": ["Delta", "Gamma", "Vega", "Theta", "Rho", "Carry Rho"],
    "Call Option": [
        bsm_call.get("delta"),
        bsm_call.get("gamma"),
        bsm_call.get("vega"),
        bsm_call.get("theta"),
        bsm_call.get("rho"),
        bsm_call.get("cRho")
    ],
    "Put Option": [
        bsm_put.get("delta"),
        bsm_put.get("gamma"),
        bsm_put.get("vega"),
        bsm_put.get("theta"),
        bsm_put.get("rho"),
        bsm_put.get("cRho")
    ]
})

print(bsm_results)

      Greeks  Call Option  Put Option
0      Delta     0.082971   -0.916550
1      Gamma     0.016823    0.016823
2       Vega     6.938711    6.938711
3      Theta    -8.126522   -1.940991
4        Rho     1.102594  -13.758003
5  Carry Rho     1.132954  -12.515272


In [6]:
# Input vector - Ensure all elements are floats
x = np.array([float(underlying), float(strike), float(ttm), float(rf), float(b), float(ivol)])

In [9]:
# Define functions for automatic differentiation
def f_call(x):
    return gbsm_greek(True, x[0], x[1], x[2], x[3], x[4], x[5]).get("price")

def f_put(x):
    return gbsm_greek(False, x[0], x[1], x[2], x[3], x[4], x[5]).get("price")

# Gradient for call and put option prices
call_grad = grad(f_call)(x)
put_grad = grad(f_put)(x)

In [10]:
# Derivative of Delta (Gamma)
def delta_call(x):
    return gbsm_greek(True, x[0], x[1], x[2], x[3], x[4], x[5], include_greeks=True).get("delta")

def delta_put(x):
    return gbsm_greek(False, x[0], x[1], x[2], x[3], x[4], x[5], include_greeks=True).get("delta")

# Gamma for call and put options
call_gamma = grad(delta_call)(x)[0]  
put_gamma = grad(delta_put)(x)[0]    

In [11]:
# Constructing the results table as a DataFrame
results_df = pd.DataFrame({
    "Greeks": ["Delta", "Gamma", "Vega", "Theta", "Rho", "CarryRho"],
    "Call Option": [
        call_grad[0],  
        call_gamma,    
        call_grad[5],  
        -call_grad[2], 
        call_grad[3],  
        call_grad[4]   
    ],
    "Put Option": [
        put_grad[0],   
        put_gamma,     
        put_grad[5],   
        -put_grad[2],  
        put_grad[3],   
        put_grad[4]    
    ]
})

# Display the DataFrame
print(results_df)

     Greeks  Call Option  Put Option
0     Delta     0.082971   -0.916550
1     Gamma     0.016823    0.016823
2      Vega     6.938711    6.938711
3     Theta    -8.126522   -1.940991
4       Rho    -0.030360   -1.242731
5  CarryRho     1.132954  -12.515272


In [12]:
def bt_american(call, underlying, strike, ttm, rf, b, ivol, N):
    
    dt = ttm / N 
    u = np.exp(ivol * np.sqrt(dt))  
    d = 1 / u  
    pu = (np.exp(b * dt) - d) / (u - d)  
    pd = 1.0 - pu  
    df = np.exp(-rf * dt)  
    z = 1 if call else -1 

    # Helper functions for node indexing
    def n_node_func(n):
        return (n + 1) * (n + 2) // 2  

    def idx_func(i, j):
        return n_node_func(j - 1) + i  

    # Total nodes in the tree
    n_nodes = n_node_func(N)
    option_values = np.empty(n_nodes)

    # Backward induction through the tree
    for j in range(N, -1, -1):  
        for i in range(j, -1, -1):  
            idx = idx_func(i, j) 
            price = underlying * (u ** i) * (d ** (j - i))  
            option_values[idx] = max(0, z * (price - strike)) 

            if j < N:  
                # Discounted value of continuation
                continuation_value = df * (pu * option_values[idx_func(i + 1, j + 1)] + pd * option_values[idx_func(i, j + 1)])
                # Take the maximum of intrinsic value and continuation value
                option_values[idx] = max(option_values[idx], continuation_value)

    return option_values[0] 

In [13]:
def bt_american_div(call, underlying, strike, ttm, rf, div_amts, div_times, ivol, N):
    
    if not div_amts or not div_times:
        return bt_american(call, underlying, strike, ttm, rf, b, ivol, N)
    elif div_times[0] > N:
        return bt_american(call, underlying, strike, ttm, rf, b, ivol, N)
    
    dt = ttm / N
    u = np.exp(ivol * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp(rf * dt) - d) / (u - d)
    pd = 1 - pu
    df = np.exp(-rf * dt)
    z = 1 if call else -1

    # Helper functions for node indexing
    def n_node_func(n):
        return (n + 1) * (n + 2) // 2

    def idx_func(i, j):
        return n_node_func(j - 1) + i

    n_div = len(div_times)
    n_nodes = n_node_func(div_times[0])
    option_values = np.zeros(n_nodes)

    for j in range(div_times[0], -1, -1):
        for i in range(j, -1, -1):
            idx = idx_func(i, j)
            price = underlying * (u ** i) * (d ** (j - i))
            
            if j < div_times[0]:
                # Times before the dividend (backward induction)
                option_values[idx] = max(0, z * (price - strike))
                option_values[idx] = max(
                    option_values[idx],
                    df * (pu * option_values[idx_func(i + 1, j + 1)] + pd * option_values[idx_func(i, j + 1)])
                )
            else:
                # Time of the dividend
                val_no_exercise = bt_american_div(
                    call,
                    price - div_amts[0],
                    strike,
                    ttm - div_times[0] * dt,
                    rf,
                    div_amts[1:],
                    [t - div_times[0] for t in div_times[1:]],
                    ivol,
                    N - div_times[0]
                )
                val_exercise = max(0, z * (price - strike))
                option_values[idx] = max(val_no_exercise, val_exercise)

    return option_values[0]

In [14]:
div_date = date(2022, 4, 11)
current_date = date(2022, 3, 13)
div_days = (div_date - current_date).days
ttm_days = int(ttm * 365)  
n_points = int(ttm_days * 3)  
div_point = div_days * 3  
div_amt = 0.88 

In [15]:
# Calculate option values
am_call = bt_american_div(True, underlying, strike, ttm, rf, [div_amt], [div_point], ivol, n_points)
am_put = bt_american_div(False, underlying, strike, ttm, rf, [div_amt], [div_point], ivol, n_points)

# Print the results
print(f"American Call Option Price: {am_call}")
print(f"American Put Option Price: {am_put}")

American Call Option Price: 0.2986340478681555
American Put Option Price: 14.55932724012951


In [16]:
def finite_difference_gradient(f, x, delta=1e-5):
    x = np.asarray(x)
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x_up = x.copy()
        x_down = x.copy()
        x_up[i] += delta
        x_down[i] -= delta
        grad[i] = (f(x_up) - f(x_down)) / (2 * delta)
    return grad

In [17]:
# Input vector - Ensure all elements are floats
x = np.array([float(underlying), float(strike), float(ttm), float(rf), float(ivol)])

# Call Option Gradients
def f_call(x):
    return bt_american_div(True, x[0], x[1], x[2], x[3], [div_amt], [div_point], x[4], n_points)

call_grad = finite_difference_gradient(f_call, x)

# Gamma for Call Option
delta = 1
call_gamma = (
    bt_american_div(True, underlying + delta, strike, ttm, rf, [div_amt], [div_point], ivol, n_points)
    + bt_american_div(True, underlying - delta, strike, ttm, rf, [div_amt], [div_point], ivol, n_points)
    - 2 * am_call
) / (delta**2)

# Dividend Sensitivity for Call Option
delta = 1e-5
call_div = (
    bt_american_div(True, underlying, strike, ttm, rf, [div_amt + delta], [div_point], ivol, n_points) - am_call
) / delta

In [18]:
# Put Option Gradients
def f_put(x):
    return bt_american_div(False, x[0], x[1], x[2], x[3], [div_amt], [div_point], x[4], n_points)

put_grad = finite_difference_gradient(f_put, x)

# Gamma for Put Option
delta = 10
put_gamma = (
    bt_american_div(False, underlying + delta, strike, ttm, rf, [div_amt], [div_point], ivol, n_points)
    + bt_american_div(False, underlying - delta, strike, ttm, rf, [div_amt], [div_point], ivol, n_points)
    - 2 * am_put
) / (delta**2)

# Dividend Sensitivity for Put Option
delta = 1e-5
put_div = (
    bt_american_div(False, underlying, strike, ttm, rf, [div_amt + delta], [div_point], ivol, n_points) - am_put
) / delta

In [19]:
am_df = pd.DataFrame({
    "Greeks": ["Delta", "Gamma", "Vega", "Theta", "Rho", "CarryRho"],
    "Call Option": [
        call_grad[0],  
        call_gamma,    
        call_grad[4],  
        -call_grad[2], 
        call_grad[3],  
        np.nan   
    ],
    "Put Option": [
        put_grad[0],   
        put_gamma,     
        put_grad[4],   
        -put_grad[2],  
        put_grad[3],   
        np.nan    
    ]
})

print(am_df)

     Greeks  Call Option  Put Option
0     Delta     0.074842   -0.935932
1     Gamma     0.018483    0.015880
2      Vega     6.319465    5.667652
3     Theta    -7.425581   -0.416989
4       Rho     0.887512  -12.334293
5  CarryRho          NaN         NaN


In [20]:
print(f"Dividend Sensitivity for Call Option: {call_div}")
print(f"Dividend Sensitivity for Put Option: {put_div}")

Dividend Sensitivity for Call Option: -0.021538913125285216
Dividend Sensitivity for Put Option: 0.9391384416801428


# Problem 2

In [21]:
# Load data
original_df_portfolio = pd.read_csv('problem2.csv')
df_portfolio = original_df_portfolio.copy()

# Load daily AAPL data
original_df_prices = pd.read_csv('DailyPrices.csv')
df_prices = original_df_prices.copy()
df_prices = df_prices[['Date', 'AAPL']]

In [22]:
underlying = 165
rf = 4.25 / 100

div_date = date(2023, 3, 15)
current_date = date(2023, 3, 3)
expiration_date = date(2023, 4, 21)
ttm = (expiration_date - current_date).days / 365
div_days = (div_date - current_date).days
ttm_days = int(ttm * 365)  
n_points = int(ttm_days * 3)  
div_point = div_days * 3 
div_amt = 1.00

In [23]:
returns = return_calculate(df_prices, method="LOG", date_column="Date")
returns = returns['AAPL']
sd = np.std(returns)

In [24]:
def calculate_implied_volatility(call, option_price, underlying, strike, ttm, rf, div_amts, div_times, N):
    # Define the objective function to find the root
    objective = lambda ivol: bt_american_div(call, underlying, strike, ttm, rf, div_amts, div_times, ivol, N) - option_price
    
    # Use Brent's method to find the implied volatility that makes the model price equal to the market price
    try:
        implied_vol = brentq(objective, 1e-5, 5)
        return implied_vol
    except ValueError:
        return np.nan  # Return NaN if the implied volatility is not found

In [25]:
# Apply the implied volatility calculation
implied_vols = []
for _, row in df_portfolio.iterrows():
    if row['Type'] == 'Option':
        call = row['OptionType'] == 'Call'
        implied_vol = calculate_implied_volatility(
            call, row['CurrentPrice'], underlying, row['Strike'],
            ttm, rf, [div_amt], [div_point], n_points 
        )
    implied_vols.append(implied_vol)

df_portfolio['Implied Volatility'] = implied_vols

In [26]:
# Define the bt_delta function
def bt_delta(call, underlying, strike, ttm, rf, div_amts, div_times, ivol, N):
    
    # Define the function to calculate option price with varying underlying
    def f(_x):
        return bt_american_div(call, _x, strike, ttm, rf, div_amts, div_times, ivol, N)
    
    # Finite difference derivative approximation for Delta
    delta_step = 1e-5  
    delta = (f(underlying + delta_step) - f(underlying - delta_step)) / (2 * delta_step)
    
    return delta

In [27]:
# Calculate Delta for Each Portfolio
portfolio_deltas = {}

for portfolio in df_portfolio['Portfolio'].unique():
    portfolio_data = df_portfolio[df_portfolio['Portfolio'] == portfolio]
    portfolio_delta = 0 

    for _, row in portfolio_data.iterrows():
        if row['Type'] == 'Option':
            call = row['OptionType'] == 'Call'
            ivol = row['Implied Volatility'] 

            # Calculate Delta for the option using bt_delta function
            delta_value = bt_delta(
                call,
                underlying,
                row['Strike'],
                ttm,
                rf,
                [div_amt],
                [div_point],
                ivol,
                n_points
            ) * row['Holding'] * underlying

            portfolio_delta += delta_value

        elif row['Type'] == 'Stock':
            portfolio_delta += row['Holding'] * underlying

    # Store the calculated delta for the portfolio
    portfolio_deltas[portfolio] = portfolio_delta

In [28]:
# Simulation parameters
nSim = 500
fwdT = 10     
sim_returns = np.random.normal(0, sd, nSim * fwdT)

In [29]:
# Collect 10-day returns
sim_prices = []
for i in range(nSim):
    r = 1.0
    for j in range(fwdT):
        r *= (1 + sim_returns[fwdT * i + j])
    sim_prices.append(underlying * r)

# Create iterations DataFrame
iterations = pd.DataFrame({'iteration': range(1, nSim + 1)})

# Cross join portfolio with iterations
values = df_portfolio.merge(iterations, how='cross')

In [30]:
# Precalculate forward time-to-maturity
def calculate_fwd_ttm(row):
    if row['Type'] == 'Option':
        return (expiration_date - current_date).days - fwdT / 365
    else:
        return None

values['fwd_ttm'] = values.apply(calculate_fwd_ttm, axis=1)

In [29]:
# Initialize result arrays
nVals = len(values)
simulated_value = np.zeros(nVals)
current_value = np.zeros(nVals)
pnl = np.zeros(nVals)

# Iterate through each row of the DataFrame
for i, row in values.iterrows():
    simulated_price = sim_prices[row['iteration'] - 1]  
    current_value[i] = row['Holding'] * row['CurrentPrice'] 
    
    if row['Type'] == 'Option':
        simulated_value[i] = row['Holding'] * bt_american_div(
            row['OptionType'] == 'Call',
            simulated_price,
            row['Strike'],
            row['fwd_ttm'],
            rf,
            [div_amt],
            [(div_days - fwdT) * 3],  
            row['Implied Volatility'],
            int(row['fwd_ttm'] * 365 * 3) 
        )
    elif row['Type'] == 'Stock':
        simulated_value[i] = row['Holding'] * simulated_price
    
    # PnL calculation
    pnl[i] = simulated_value[i] - current_value[i]

# Add calculated columns back to the DataFrame
values['simulatedValue'] = simulated_value
values['pnl'] = pnl
values['currentValue'] = current_value

KeyboardInterrupt: 

In [33]:
# Convert portfolio_deltas dictionary to a DataFrame
portfolio_deltas_df = pd.DataFrame(list(portfolio_deltas.items()), columns=['Portfolio', 'PortfolioDelta'])

# Parameters for VaR and ES calculations
quantile_05 = norm.ppf(0.05)  
sqrt_10 = np.sqrt(10)  

# Add Mean (Portfolio Delta * Current Underlying Price)
portfolio_deltas_df['Mean'] = portfolio_deltas_df['PortfolioDelta']

# Delta-Normal VaR as $ loss
portfolio_deltas_df['DN_VaR'] = abs(
    quantile_05 * sqrt_10 * sd * portfolio_deltas_df['PortfolioDelta'] * underlying
)

# Delta-Normal ES as $ loss
portfolio_deltas_df['DN_ES'] = abs(
    (sqrt_10 * sd * norm.pdf(quantile_05) / 0.05) * portfolio_deltas_df['PortfolioDelta'] * underlying
)

# Display results
print(portfolio_deltas_df)

      Portfolio  PortfolioDelta          Mean        DN_VaR         DN_ES
0      Straddle    1.201974e+02  1.201974e+02  1.452452e+03  1.821434e+03
1       SynLong    2.098026e+02  2.098026e+02  2.535232e+03  3.179283e+03
2    CallSpread    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
3     PutSpread   -2.241398e+01 -2.241398e+01  2.708480e+02  3.396544e+02
4         Stock    1.650000e+02  1.650000e+02  1.993842e+03  2.500358e+03
5         Call     1.650000e+02  1.650000e+02  1.993842e+03  2.500358e+03
6          Put    -4.480259e+01 -4.480259e+01  5.413896e+02  6.789244e+02
7   CoveredCall   -5.237325e-08 -5.237325e-08  6.328727e-07  7.936478e-07
8  ProtectedPut    1.360388e+02  1.360388e+02  1.643878e+03  2.061489e+03


# Problem 3

In [31]:
# Load data
ff_R = pd.read_csv("F-F_Research_Data_Factors_daily.csv")
ff_M = pd.read_csv("F-F_Momentum_Factor_daily.csv")
prices = pd.read_csv("DailyPrices.csv")

In [32]:
rf = 0.05
returns = return_calculate(prices, method="LOG", date_column="Date")

# Convert the Date column to string type to handle any unexpected formats
ff_R['Date'] = ff_R['Date'].astype(str)
ff_M['Date'] = ff_M['Date'].astype(str)

# Ensure only date rows are processed by filtering out non-date entries
ff_R = ff_R[ff_R['Date'].str.isnumeric()]
ff_M = ff_M[ff_M['Date'].str.isnumeric()]

In [33]:
# Convert Date columns and adjust factor returns
ff_R['Date'] = pd.to_datetime(ff_R['Date'], format='%Y%m%d')
ff_M['Date'] = pd.to_datetime(ff_M['Date'], format='%Y%m%d')
ff_R = ff_R.rename(columns={ff_R.columns[-1]: 'Mom'}) 
ff_R.iloc[:, 1:] /= 100  

# Merge Fama-French 3-factor data with Momentum data
ff = pd.merge(ff_R, ff_M, on='Date')
returns['Date'] = pd.to_datetime(returns['Date']).dt.normalize()

In [34]:
# List of stocks
stocks = ['AAPL', 'META', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE', 'AMZN', 'BRK-B', 
          'PG', 'XOM', 'TSLA', 'JPM', 'V', 'DIS', 'GOOGL', 'JNJ', 'BAC', 'CSCO']

# Join stock returns with the factors data
data = pd.merge(returns[['Date'] + stocks], ff, on='Date')

In [35]:
# Regression for each stock to find betas
x_names = ["Mkt-RF", "SMB", "HML", "Mom"]
X = add_constant(data[x_names])
Y = data[stocks]

# Calculate betas for each stock using OLS
betas = np.array([OLS(Y[col], X).fit().params.values for col in Y.columns])

In [36]:
# Calculate the means of the last 10 years of factor returns
ff_10yr = ff[ff['Date'] >= '2014-09-30']
means = np.array([0.0] + list(ff_10yr[x_names].mean()))

# Calculate expected annual returns and covariance matrix
stock_mean = np.log(1 + betas @ means) * 365 + rf
log_returns = np.log(1 + data[stocks])
covar = log_returns.cov() * 365

In [37]:
# Define the Sharpe ratio function for maximization
def sharpe_ratio(weights, stock_mean, covar, rf):
    portfolio_return = np.dot(weights, stock_mean)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covar, weights)))
    return -((portfolio_return - rf) / portfolio_volatility)  

In [38]:
# Optimization to find super-efficient portfolio weights
n = len(stocks)
initial_weights = np.array([1 / n] * n)

# Constraints: sum of weights = 1 and each weight >= 0
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = [(0, 1) for _ in range(n)]

# Perform optimization
result = minimize(sharpe_ratio, initial_weights, args=(stock_mean, covar, rf), method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimized weights
opt_weights = result.x.round(4)

In [39]:
# Display results
opt_df = pd.DataFrame({"Stock": stocks, "Weight": opt_weights, "Return": stock_mean})

# Print outputs
print(opt_df)
print("Expected Return =", np.dot(stock_mean, opt_weights))
print("Expected Volatility =", np.sqrt(np.dot(opt_weights.T, np.dot(covar, opt_weights))))
print("Expected Sharpe Ratio =", -result.fun)  

    Stock  Weight    Return
0    AAPL  0.0000 -2.253823
1    META  0.0026  1.172346
2     UNH  0.0000 -0.114615
3      MA  0.0000  1.773019
4    MSFT  0.0000 -0.368813
5    NVDA  0.0000 -3.702442
6      HD  0.1057  3.037572
7     PFE  0.0000 -5.385713
8    AMZN  0.0000 -0.014672
9   BRK-B  0.1501  2.170457
10     PG  0.4020  2.943831
11    XOM  0.0000  1.502708
12   TSLA  0.0000 -4.043305
13    JPM  0.0000  0.183736
14      V  0.0000  0.364972
15    DIS  0.0530  2.259346
16  GOOGL  0.0000 -3.157294
17    JNJ  0.0653  1.943940
18    BAC  0.0000  0.016937
19   CSCO  0.2213  3.712624
Expected Return = 2.9016135536959844
Expected Volatility = 0.12554491218852082
Expected Sharpe Ratio = 22.713892061420275
